In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cross_validation import train_test_split,KFold
from sklearn.ensemble import ExtraTreesClassifier,GradientBoostingClassifier
from sklearn import preprocessing, ensemble
from sklearn.metrics import log_loss
import xgboost as xgb
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
from keras.layers.core import Dense, Activation, Merge, Flatten, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.models import Sequential
import keras
from PIL import Image
import requests
from io import BytesIO
from skimage import measure, feature, color, filters, draw, segmentation, morphology, exposure, transform
from StringIO import StringIO
from skimage import transform as tf
from sklearn.cross_validation import cross_val_predict,cross_val_score
%matplotlib inline

stoppers=stopwords.words('english')

In [2]:
train=pd.read_json('/Users/thomas/Data Science Projects/Two_Sigma/Files/train.json')
test=pd.read_json('/Users/thomas/Data Science Projects/Two_Sigma/Files/test.json')

In [3]:
def clean_street_address(address,street_block=True):
    new_address=""
    temp=address.upper()
    if "-" in temp:
        temp=temp.replace('-'," ")
    split_address=temp.split()
    if len(split_address)<1:
        return ""
    if len(split_address)==1:
        return split_address[0].upper()
    else:
        temp=[]
        for x in split_address:
            if x!="":
                temp.append(x)
        split_address=temp
        number=""
        if len(split_address)==2:
            pass
        else:
            number=split_address[0][:-2]
            split_address=split_address[1:]
        if len(split_address[0])>2:
            if (split_address[0][-2:]=='ST') or (split_address[0][-2:]=='ND') or (split_address[0][-2:]=='RD'):
                try:
                    int(split_address[0][:-2])
                    split_address[0]=split_address[0][:-2]
                except:
                    pass
        if (split_address[-1].lower()!='w') or (split_address[-1].lower()!='e'):
            if (split_address[-1][:2].lower()=='av') or (split_address[-1][:2].lower()=='st') or \
            (split_address[-1][:2].lower()=='bl') or (split_address[-1][:2].lower()=='pl'):
                split_address[-1]=split_address[-1][:2]
        
        for x in split_address:
            new_address+=x.upper()+" "
        new_address=new_address[:-1]
        new_address=new_address.replace('FIRST','1').replace('SECOND','2').replace('THIRD','3').replace('FOURTH','4')\
                    .replace('FIFTH','5').replace('SIXTH','6').replace('SEVENTH','7').replace('EIGTH','8')\
                    .replace('NINTH','9').replace('TENTH','10').replace('EAST','E').replace('WEST','W')\
                    .replace('E.','E').replace('W.','W')
        if street_block:
            if number!="":
                new_address=number+" "+new_address
        return new_address

In [4]:
manager_id_dict_train=train.groupby(['manager_id'])['manager_id'].count().to_frame().sort_values(by='manager_id',\
                                        ascending=False).to_dict()['manager_id']
building_id_dict_train=train.groupby(['building_id'])['building_id'].count().to_frame().sort_values(by='building_id',\
                                        ascending=False).to_dict()['building_id']
train['manager_property_count']=train['manager_id'].apply(lambda x: manager_id_dict_train[x])
train['building_listing_count']=train['building_id'].apply(lambda x: building_id_dict_train[x])
train['photos']=train['photos'].apply(lambda x: len(x))
train['dayofyear']=train['created'].apply(lambda x: pd.to_datetime(x).dayofyear)
train['street_address_with_block']=train['street_address'].apply(lambda x: clean_street_address(x))
train['street_address']=train['street_address'].apply(lambda x: clean_street_address(x,street_block=False))
columns=[u'bathrooms', u'bedrooms',u'features',u'interest_level',u'price','latitude','longitude',\
      'street_address_with_block',u'street_address','dayofyear','photos','building_listing_count',\
         'manager_property_count','description']
train=train[columns]




manager_id_dict_test=test.groupby(['manager_id'])['manager_id'].count().to_frame().sort_values(by='manager_id',\
                                        ascending=False).to_dict()['manager_id']
building_id_dict_test=test.groupby(['building_id'])['building_id'].count().to_frame().sort_values(by='building_id',\
                                        ascending=False).to_dict()['building_id']
test['manager_property_count']=test['manager_id'].apply(lambda x: manager_id_dict_test[x])
test['building_listing_count']=test['building_id'].apply(lambda x: building_id_dict_test[x])

test['photos']=test['photos'].apply(lambda x: len(x))
test['street_address_with_block']=test['street_address'].apply(lambda x: clean_street_address(x))
test['street_address']=test['street_address'].apply(lambda x: clean_street_address(x,street_block=False))
test['dayofyear']=test['created'].apply(lambda x: pd.to_datetime(x).dayofyear)
test['street_address_with_block']=test['street_address'].apply(lambda x: clean_street_address(x))
test['street_address']=test['street_address'].apply(lambda x: clean_street_address(x,street_block=False))
columns=[u'bathrooms', u'bedrooms',u'features',u'price','latitude','longitude','street_address_with_block',\
         u'street_address','dayofyear','photos','building_listing_count','manager_property_count','description']
test=test[columns]

In [5]:
for row in train.index:
    features=train.loc[row,'features']
    x=""
    for feature in features:
        x+=feature.upper().replace('-',"").replace('.'," ").replace("  "," ")+" "
    x=x[:-1]
    train.loc[row,'features']=x

print "Done 1"
feature_dict={}
for row in train.index:
    for temp in train.loc[row,'features'].split():
        if temp not in stoppers:
            if temp in feature_dict.keys():
                feature_dict[temp]+=1
            else:
                feature_dict[temp]=1
print "2",       
keep_list=[]
for x in feature_dict.keys():
    if feature_dict[x]>30:
        keep_list.append(x)

print "3",
def feature_finder(features_list,feature_to_find):
    if feature_to_find in features_list:
        return 1
    else:
        return 0

for feature in keep_list:
    train[str(feature)]=train['features'].apply(lambda x: feature_finder(x,feature))
del train['features']
print "4",

# ### TESTING

for row in test.index:
    features=test.loc[row,'features']
    x=""
    for feature in features:
        x+=feature.upper().replace('-',"").replace('.'," ").replace("  "," ")+" "
    x=x[:-1]
    test.loc[row,'features']=x

print "5",
for feature in keep_list:
    test[str(feature)]=test['features'].apply(lambda x: feature_finder(x,feature))
del test['features']
print "6"

Done 1
2 3 4 5 6


In [6]:
for row in train.index:
    description=train.loc[row,'description']
    description=description.upper().replace('-',"").replace('.'," ").replace("  "," ")
    train.loc[row,'description']=description

print "Done 1",
description_dict={}
for row in train.index:
    choice_number=12
    split_list=train.loc[row,'description'].split()
    if len(split_list)<choice_number:
        choice_number=len(split_list)
    if len(split_list)>0:
        for temp in np.random.choice(split_list,choice_number):
            if temp not in stoppers:
                if temp in description_dict.keys():
                    description_dict[temp]+=1
                else:
                    description_dict[temp]=1

print "2",
description_keep_list=[]
for x in description_dict.keys():
    if description_dict[x]>30:
        description_keep_list.append(x)
        
print "3",
def feature_finder(features_list,feature_to_find):
    if feature_to_find in features_list:
        return 1
    else:
        return 0

for description in description_keep_list:
    train[str(description)]=train['description'].apply(lambda x: feature_finder(x,description))
del train['description']
print "4",

#--------------------------

for row in test.index:
    description=test.loc[row,'description']
    description=description.upper().replace('-',"").replace('.'," ").replace("  "," ")
    test.loc[row,'description']=description

print "5",
for description in description_keep_list:
    test[str(description)]=test['description'].apply(lambda x: feature_finder(x,description))
del test['description']
print "6"

Done 1 2 3 4 5 6


# XG-Boost

In [7]:
Y=train['interest_level'].copy()
X=train.copy()
del X['interest_level']
del X['street_address']
del X['street_address_with_block']
X=X.reset_index(drop=True)
X=sparse.csr_matrix(X)

Y=[{'high':0, 'medium':1, 'low':2}[x] for x in Y]
x_train,x_test,y_train,y_test=train_test_split(X,Y,random_state=1)


X_testing_set=test.copy()
del X_testing_set['street_address']
del X_testing_set['street_address_with_block']
X_testing_set=sparse.csr_matrix(X_testing_set)

In [71]:
param = {}
param['objective'] = 'multi:softprob' 
param['eta'] = 0.055 # 0.05
param['max_depth'] = 6 # 6
param['silent'] = 1
param['num_class'] = 3
param['eval_metric'] = "mlogloss"
param['min_child_weight'] = 2       # 2
param['subsample'] = 0.5            # .5  
param['colsample_bytree'] = .75      # 1. 
param['alpha'] =0.3
param['lambda']=0.3
# param['nfold']=10
param['seed']=0
# param['stratified'] = True

plst = list(param.items())

xgtrain = xgb.DMatrix(X, label=Y)
xgtest = xgb.DMatrix(X_testing_set)
watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]

model=xgb.cv(plst, xgtrain, 4000,  early_stopping_rounds=50,nfold=6, stratified=True, verbose_eval=True)
# preds = model.predict(xgtest)

[0]	train-mlogloss:1.06516+0.00107869	test-mlogloss:1.06563+0.00119944
[1]	train-mlogloss:1.03451+0.00135027	test-mlogloss:1.03548+0.00146325
[2]	train-mlogloss:1.00656+0.00153993	test-mlogloss:1.00807+0.00153443
[3]	train-mlogloss:0.981698+0.00114853	test-mlogloss:0.983715+0.00109241
[4]	train-mlogloss:0.957902+0.000797929	test-mlogloss:0.960411+0.000871496
[5]	train-mlogloss:0.936316+0.00154734	test-mlogloss:0.939364+0.00145798
[6]	train-mlogloss:0.916515+0.00179143	test-mlogloss:0.920043+0.00146631
[7]	train-mlogloss:0.897861+0.00205878	test-mlogloss:0.901804+0.00182723
[8]	train-mlogloss:0.880596+0.00212786	test-mlogloss:0.884985+0.00210806
[9]	train-mlogloss:0.864523+0.00240077	test-mlogloss:0.86941+0.00217645
[10]	train-mlogloss:0.849701+0.00238758	test-mlogloss:0.855113+0.00216681
[11]	train-mlogloss:0.835807+0.00210031	test-mlogloss:0.841664+0.00164363
[12]	train-mlogloss:0.821916+0.00177978	test-mlogloss:0.828226+0.00142167
[13]	train-mlogloss:0.809487+0.00167922	test-mlogloss

# BEST 
train-mlogloss:0.374232	test-mlogloss:0.5696

--------------------------------

train-mlogloss:0.409219	test-mlogloss:0.572398
[609]	train-mlogloss:0.371017	test-mlogloss:0.570851 - 0.050
[504]	train-mlogloss:0.393776	test-mlogloss:0.57051  - 0.051
[453]	train-mlogloss:0.399076	test-mlogloss:0.570283 - 0.055

train-mlogloss:0.466407	test-mlogloss:0.583448 -> 10 (ETA 0.075)
train-mlogloss:0.466549	test-mlogloss:0.583876 -> 10 (ETA 0.07)
train-mlogloss:0.481274	test-mlogloss:0.584468 -> 10 (ETA 0.06)
train-mlogloss:0.464393	test-mlogloss:0.584485 -> 10 (ETA 0.05)

 -> 25 (ETA 0.066)

train-mlogloss:0.452309	test-mlogloss:0.582809 -> 30 (ETA 0.0635)
train-mlogloss:0.453463	test-mlogloss:0.582848 -> 30 (ETA 0.0628)
train-mlogloss:0.438918	test-mlogloss:0.582536 -> 30 (ETA 0.0626) --<--
train-mlogloss:0.452579	test-mlogloss:0.582686 -> 30 (ETA 0.0625)
train-mlogloss:0.44286	test-mlogloss:0.583615 -> 30 (ETA 0.0624)
train-mlogloss:0.470476	test-mlogloss:0.583641 -> 30 (ETA 0.06)

train-mlogloss:0.456935	test-mlogloss:0.583707 -> 50 (ETA 0.05)

# END HERE

In [ ]:
param = {}
param['objective'] = 'multi:softprob' 
param['eta'] = 0.055 # 0.05
param['max_depth'] = 6 # 6
param['silent'] = 1
param['num_class'] = 3
param['eval_metric'] = "mlogloss"
param['min_child_weight'] = 2       # 2
param['subsample'] = 0.5            # .5  
param['colsample_bytree'] = .75      # 1.   

plst = list(param.items())

xgtrain = xgb.DMatrix(x_train, label=y_train)
xgtest = xgb.DMatrix(x_test, label=y_test)
watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]

model=xgb.train(plst, xgtrain, 4000, watchlist, early_stopping_rounds=50)
preds = model.predict(xgtest)

train-mlogloss:0.483514	test-mlogloss:0.581369

train-mlogloss:0.468113	test-mlogloss:0.580092

train-mlogloss:0.478416	test-mlogloss:0.580315

train-mlogloss:0.491241	test-mlogloss:0.583332

train-mlogloss:0.478357	test-mlogloss:0.581878

train-mlogloss:0.458215	test-mlogloss:0.583713

train-mlogloss:0.470414	test-mlogloss:0.582421 -> 6

train-mlogloss:0.498787	test-mlogloss:0.584362 -> 5

train-mlogloss:0.511089	test-mlogloss:0.585577 -> 4

In [11]:
param = {}
param['objective'] = 'multi:softprob' 
param['eta'] = 0.055 # 0.05
param['max_depth'] = 6 # 6
param['silent'] = 1
param['num_class'] = 3
param['eval_metric'] = "mlogloss"
param['min_child_weight'] = 2       # 2
param['subsample'] = 0.5            # .5  
param['colsample_bytree'] = .75      # 1. 
param['alpha'] =0.3
param['lambda']=0.3  

plst = list(param.items())

xgtrain = xgb.DMatrix(X, label=Y)
xgtest = xgb.DMatrix(X_testing_set)

model=xgb.train(plst, xgtrain, 540)
preds = model.predict(xgtest)

In [12]:
temp=pd.read_json('/Users/thomas/Data Science Projects/Two_Sigma/Files/test.json')
proba_df = pd.DataFrame(preds)
proba_df.columns = ["high", "medium", "low"]
proba_df["listing_id"] = temp['listing_id'].values
proba_df.to_csv("/Users/thomas/Data Science Projects/Two_Sigma/Two_Sigma_Log_Loss.csv", index=False)

# Neural Network

In [ ]:
Y=train['interest_level'].copy()
X=train.copy()
del X['interest_level']
x_train,x_test,y_train,y_test=train_test_split(np.array(X),np.array(Y),random_state=1)

nn_y_train=np.array(pd.get_dummies(y_train))
nn_y_test=np.array(pd.get_dummies(y_test))
X_testing_set=np.array(test.copy())

In [ ]:
dense_layers=[202,202,202]
model=Sequential()
model.add(Dense(202, input_dim=x_train.shape[1],activation='relu'))
for dl in dense_layers:
    model.add(Dense(dl, activation='relu'))
    model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))
sgd=keras.optimizers.SGD(lr=0.000001,decay=0.000000,momentum=.0)
model.compile(loss='categorical_crossentropy',optimizer='adamax',metrics=['accuracy'])

In [ ]:
model.fit(x_train,nn_y_train,batch_size=250,nb_epoch=5,shuffle=True,verbose=1,validation_data=(x_test,nn_y_test))

In [ ]:
proba=model.predict_proba(X_testing_set)

In [ ]:
proba[2]

In [ ]:
print np.shape(x_train)
np.shape(nn_y_train)

In [ ]:
x_train_images_list_left=np.array([np.reshape(image[0],(3, 125, 125)) for image in x_train_images_list])
x_train_images_list_right=np.array([np.reshape(image[1],(3, 125, 125)) if len(image)>1 else \
                np.reshape(image[0],(3, 125, 125)) for image in x_train_images_list])
x_test_images_list_left=np.array([np.reshape(image[0],(3, 125, 125)) for image in x_test_images_list])
x_test_images_list_right=np.array([np.reshape(image[1],(3, 125, 125)) if len(image)>1 else \
                np.reshape(image[0],(3, 125, 125)) for image in x_test_images_list])

In [ ]:
# conv_layers=[32, 32]
# dense_layers=[64,64]

# model_left=Sequential()
# model_left.add(Convolution2D(32,3,3, activation='relu', input_shape=(3,125,125), dim_ordering="th"))
# model_left.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))
# for layer in conv_layers:
#     model_left.add(Convolution2D(layer, 3, 3, activation='relu', dim_ordering="th"))
#     model_left.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="tf"))
# model_left.add(Flatten())

# model_right=Sequential()
# model_right.add(Convolution2D(32,3,3, activation='relu', input_shape=(3,125,125), dim_ordering="th"))
# model_right.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))
# for layer in conv_layers:
#     model_right.add(Convolution2D(layer, 3, 3, activation='relu', dim_ordering="th"))
#     model_right.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="tf"))
# model_right.add(Flatten())

# merged=Merge([model_left,model_right],mode='concat')

# model=Sequential()
# model.add(merged)
# for dl in dense_layers:
#     model.add(Dense(dl, activation='relu'))
#     model.add(Dropout(0.5))
# model.add(Dense(3, activation='softmax'))
# sgd=keras.optimizers.SGD(lr=0.4,decay=0.0002,momentum=.1)
# model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])

In [ ]:
# model.fit([x_train_images_list_left,x_train_images_list_right],\
#           [[1,0,0] if x==0 else [0,1,0] if x==1 else [0,0,1] for x in y_train],\
#           batch_size=20,nb_epoch=3,shuffle=True,verbose=1,\
#           validation_data=([x_test_images_list_left,x_test_images_list_right],\
#             [[1,0,0] if x==0 else [0,1,0] if x==1 else [0,0,1] for x in y_test]))

In [ ]:
# model.predict_proba([x_test_images_list_left,x_test_images_list_right])